In [1]:
from generator import *
import pandas as pd
from datasets import load_dataset

In [14]:
add_entries("./prompts", "dataset/output.csv")
clean_entries("dataset/output.csv")
no_tone_csv("dataset/output.csv")
target_to_nb("dataset/no_tone_output.csv")
print("Files were created")

Files were created


In [4]:
# if there are issues 
#! python generator.py

In [15]:
data = pd.read_csv("dataset/no_tone_nbtarget_output.csv")
print(data.shape)
data.head()

(668, 2)


,target,text
0,0,have you seen how incompetent middle easterner...
1,0,"they have been invaded so many times, they hav..."
2,0,civilization has not reached the middle east yet
3,0,what the middle east needs is a good invasion
4,0,"the middle easterners are animals, they kill a..."


In [8]:
data.isna().sum()

target    0
text      0
dtype: int64

In [12]:
print(int(round(data.shape[0]/15,0)))

11961


In [16]:
test = data.sample(int(round(data.shape[0]/15,0)))
test.to_csv("dataset/test.csv", index=False)

data_files = {"train": "no_tone_nbtarget_output.csv", "test": "test.csv"}
dataset = load_dataset("dataset", data_files=data_files)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [26]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['target', 'text'],
        num_rows: 668
    })
    test: Dataset({
        features: ['target', 'text'],
        num_rows: 20
    })
})


In [27]:
from transformers import AutoTokenizer

tokenizer  = AutoTokenizer.from_pretrained("google/bert_uncased_L-2_H-128_A-2")

In [28]:
print(dataset["train"])
print(dataset["train"][0]["text"])
# print(tokenizer(dataset["train"][0]["text"]))

Dataset({
    features: ['target', 'text'],
    num_rows: 668
})
have you seen how incompetent middle easterners are? their schools are so much worse than ours!


In [29]:
def preprocess_text_classification(example):
    encoded_example           = tokenizer(example["text"], max_length=512)
    encoded_example["target"] = example["target"]
    return encoded_example

dataset_tok = dataset.map(
    preprocess_text_classification
)

Map:   0%|          | 0/668 [00:00<?, ? examples/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [30]:
print(dataset_tok)

DatasetDict({
    train: Dataset({
        features: ['target', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 668
    })
    test: Dataset({
        features: ['target', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 20
    })
})


In [31]:
dataset_tok = dataset_tok.remove_columns(["text"])
dataset_tok = dataset_tok.rename_column("target", "labels")

In [32]:
dataset_tok.set_format("torch")

In [33]:
train = dataset_tok["train"].shuffle(seed=42)
test = dataset_tok["test"].shuffle(seed=42)

In [34]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train, shuffle=True)
eval_dataloader = DataLoader(test)

In [35]:
id2label = {0: "NEGATIVE", 
            1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [44]:
id2label = {i: data.target.unique()[i] for i in range(len(data.target.unique()))}
# label2id = {data.target.unique()[i]: i for i in range(len(data.target.unique()))}
label2id = {'middle_east': 0,'latino': 1,'chinese': 2,'muslim': 3,'bisexual': 4,'mexican': 5,'lgbtq': 6,'physical_disability': 7,'mental_disability': 8,'asian': 9,'women': 10,'jewish': 11,'immigrant': 12,'native_american': 13,'black': 14}
id2label = {k:v for (k,v) in enumerate(label2id)}

print(label2id)
print(id2label)


{'middle_east': 0, 'latino': 1, 'chinese': 2, 'muslim': 3, 'bisexual': 4, 'mexican': 5, 'lgbtq': 6, 'physical_disability': 7, 'mental_disability': 8, 'asian': 9, 'women': 10, 'jewish': 11, 'immigrant': 12, 'native_american': 13, 'black': 14}
{0: 'middle_east', 1: 'latino', 2: 'chinese', 3: 'muslim', 4: 'bisexual', 5: 'mexican', 6: 'lgbtq', 7: 'physical_disability', 8: 'mental_disability', 9: 'asian', 10: 'women', 11: 'jewish', 12: 'immigrant', 13: 'native_american', 14: 'black'}


In [45]:
from transformers import AutoModelForSequenceClassification
# BERT-tiny license

# Copyright 2018 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


model_bert_l4 = AutoModelForSequenceClassification.from_pretrained(
    "google/bert_uncased_L-4_H-128_A-2", num_labels=len(data.target.unique()), id2label=id2label, label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-4_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
print("bert-tiny variant number of parameters: ", model_bert_l4.num_parameters())

bert-tiny variant number of parameters:  4784399


In [47]:
from torch.optim import AdamW

optimizer = AdamW(model_bert_l4.parameters(), lr=5e-5)

In [48]:
from transformers import get_scheduler

num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader)
# feel free to experiment with different num_warmup_steps
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=1, num_training_steps=num_training_steps
)

In [49]:
Gustave Besacierimport torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_bert_l4.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, e

In [50]:
print(train_dataloader)
for i in train_dataloader:
    a, b, c, d = i.items()
print(a)
print(b)
print(c)
print(d)

('labels', tensor([6]))
('input_ids', tensor([[  101,  4190, 25090,  4328,  6774,  1996, 19483,  9580,  2097,  2599,
         21268,  2000,  2019,  3623,  1999,  4126,   102]]))
('token_type_ids', tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]))
('attention_mask', tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]))


In [51]:
for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
batch

{'labels': tensor([12]),
 'input_ids': tensor([[  101,  1996,  1043, 22571, 14625,  1010,  2572,  1045,  2157,  1029,
           2122,  4365,  3097,  2308,  2272,  2074,  2000,  8954,  2256, 19089,
           2005,  2037,  2769,   102]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [52]:
# use the tqdm library to add a progress bar over the number of training steps
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

# put the model in train mode
model_bert_l4.train()

# iterate over epochs
for epoch in range(num_epochs):
    # iterate over batches in training set
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model_bert_l4(**batch)
        
        loss = outputs.loss

        loss.backward()

        optimizer.step()
        
        lr_scheduler.step()

        optimizer.zero_grad()

        progress_bar.update(1)

  0%|          | 0/1336 [00:00<?, ?it/s]

In [63]:
text_asian = "Go back to your country"
text_femal = "You are a female"
text_pos_neg = [text_asian, text_femal]

In [64]:

inputs = tokenizer(text_asian, return_tensors="pt",  padding = True, truncation = True)

inputs = inputs.to(device)

with torch.no_grad():
    
    outputs = model_bert_l4(**inputs)

    logits = outputs.logits



predicted_class_id = logits.argmax().item()
print("predicted_class_id: ", predicted_class_id)


print(model_bert_l4.config.id2label[predicted_class_id])

inputs = tokenizer(text_femal, return_tensors="pt",  padding = True, truncation = True)


with torch.no_grad():
    logits = model_bert_l4(**inputs.to(device)).logits

predicted_class_id = logits.argmax().item()
print("predicted_class_id: ", predicted_class_id)

print(model_bert_l4.config.id2label[predicted_class_id])

predicted_class_id:  12
immigrant
predicted_class_id:  6
lgbtq


In [80]:
prompt = input("Prompt: ")
inputs = tokenizer(prompt, return_tensors="pt",  padding = True, truncation = True)

inputs = inputs.to(device)

with torch.no_grad():
    
    outputs = model_bert_l4(**inputs)

    logits = outputs.logits


predicted_class_id = logits.argmax().item()

print("predicted_class_id: ", predicted_class_id)
print("target category", model_bert_l4.config.id2label[predicted_class_id])

Prompt:  Gay people are often blamed


predicted_class_id:  6
target category lgbtq
